# Example of using QuOCS with Qudi

We want to run an optimization on one of our test-problems provided in QuOCS. TO demonstrate some of the advantages of QuOCS in experiment optimization, we add noise to the figure of merit (FoM) and make use of the re-evaluation steps functionality (see below). But let's go through the setup of the optimization step-py-step.

## Import statements

We need to import the QuOCS Optimizer and the test-problem (OneQubitProblem). Let's also include numpy for good measure becuase we might need it later.

In [1]:
from quocslib.Optimizer import Optimizer
# from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
from quocslib.optimalcontrolproblems.OneQubitProblem import OneQubit
import numpy as np
import time

## The Configuration of QuOCS

To configure QuOCS, you need to fill the ```optimization_dictionary``` with information. In this example, we will do it directly in the code, but you can also load the dictionary from a JSON file.

Let's define the name of the optimization. This will also show up in the name of the folder containing the results.

In [2]:
optimization_client_name = "dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name}

Now we need to specify the general settings, such as the used algorithm (in this case dCRAB) and its parameters (e.g. number of super-iterations). In the same section we choos to use the re-evaluation steps method for noisy optimization. Extending the dCRAB algorithm, the direct dearch method (dsm) needs to be specified and stopping criteria for each sub-iteration can be defined.

In [3]:
optimization_dictionary["algorithm_settings"] = {
        "algorithm_name": "dCRAB",
        "super_iteration_number": 5,
        "max_eval_total": 5000,
        "re_evaluation": {
            "re_evaluation_steps": [0.3, 0.5, 0.51]
        },
        "dsm_settings": {
            "general_settings": {
                "dsm_algorithm_name": "NelderMead",
                "is_adaptive": True
            },
            "stopping_criteria": {
                "xatol": 1e-2,
                "frtol": 1e-2,
                "max_eval": 200,
                "change_based_stop": {
                    "cbs_funct_evals": 50,
                    "cbs_change": 0.05
                }
            }
        }
    }

### Times

The time of a pulse is specified separately to use it as an optimization parameter in future updates and so that different pulses can act on separate timescales but can be grouped by common durations. Since there can me more than one time, the entries have to be provided as a list (note the square brackets around the dictionary for ```time_1```):

In [4]:
optimization_dictionary["times"] = [{
        "time_name": "time_1",
        "initial_value": 3.0
    }]

### Parameters

Since we do not have any parameters in this example, we just add an empty list for the settings of parameter optimization.

In [5]:
optimization_dictionary['parameters'] = []

### Pulses

Now we define a pulse to be expanded in the Fourier basis and some other attributes:

In [6]:
pulse_1 = {"pulse_name": "Pulse_1",
           "upper_limit": 15.0,
           "lower_limit": -15.0,
           "bins_number": 101,
           "time_name": "time_1",
           "amplitude_variation": 0.3,
           "basis": {
               "basis_name": "Fourier",
               "basis_vector_number": 2,
               "random_super_parameter_distribution": {
                   "distribution_name": "Uniform",
                   "lower_limit": 0.1,
                   "upper_limit": 5.0
               }
           },
           "scaling_function": {
               "function_type": "lambda_function",
               "lambda_function": "lambda t: 1.0 + 0.0*t"
           },
           "initial_guess": {
               "function_type": "lambda_function",
               "lambda_function": "lambda t: np.pi/3.0 + 0.0*t"
           }
    }

And we instert it into the list of pulses for the settings dict:

In [7]:
optimization_dictionary['pulses'] = [pulse_1]

Finally, we define some parameters for the FoM object:

In [8]:
args_dict = {
        "initial_state": "[1.0 , 0.0]",
        "target_state": "[1.0/np.sqrt(2), -1j/np.sqrt(2)]",
        "is_noisy": True,
        "noise_factor": 0.05
    }

### Now let's put it all together and start the optimization

Load the optimization algorithm into the optimization logic and display it into the GUI

In [9]:
optimizationlogic.load_opti_comm_dict({"optimization_dictionary": optimization_dictionary})

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stands for the experimental standard deviation.

In [10]:
# define the FoM object
FoM_object = OneQubit(args_dict=args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    # Wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            

    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    
    
    # Calculate the Figure of Merit
    fom_dict = FoM_object.get_FoM(pulses, parameters, timegrids)
    
    # Extract the fom and std
    fom, std = fom_dict["FoM"], fom_dict["std"]
    
    # Update the figure of merit and the standard deviation to the FoM logic
    fomlogic.update_fom(fom, std, status_code=0)
    
    # Update the last time the FoM is calculated
    last_time_fom = time.time()
    
    # Print the data just for debug purpose
    print("FoM: {fom}, Std: {std}".format(fom=fom, std=std))
print("Optimization finished")

FoM: 0.6584645639445008, Std: 0.008535731061605122
FoM: 0.6904611377628531, Std: 0.0032021840236500644
FoM: 0.698748976993722, Std: 0.00421047201226274
FoM: 0.585228496302175, Std: 0.0020439250781367113
FoM: 0.66742784777252, Std: 0.0015460417496038392
FoM: 0.6621773589057096, Std: 0.0016058183346016885
FoM: 0.633767141165585, Std: 0.006362211020008187
FoM: 0.6474730357817642, Std: 0.008363285241852356
FoM: 0.6391089939855978, Std: 0.004729637440769948
FoM: 0.6111248063065106, Std: 0.007588148163350421
FoM: 0.6307015031059348, Std: 0.002997689422269754
FoM: 0.6866278305725656, Std: 0.00999710967479298
FoM: 0.5823517283966282, Std: 0.004678961076252832
FoM: 0.6034634328310654, Std: 0.004606283521492351
FoM: 0.6280131465130994, Std: 0.008755230333401028
FoM: 0.5611584968022318, Std: 0.009463579050563503
FoM: 0.5995433786074924, Std: 0.0016566206945279284
FoM: 0.6120982192499698, Std: 0.0011433008159386227
FoM: 0.5900488389637965, Std: 0.0029674634489479834
FoM: 0.6179557634908183, Std: 0

FoM: 0.042281856136854146, Std: 0.004566153942853189
FoM: 0.045011018239388106, Std: 0.0055540883201658665
FoM: 0.043460905187480206, Std: 0.0027314657431847978
FoM: 0.046125244142147394, Std: 0.004096378836911794
FoM: 0.030734517874698677, Std: 0.006436887503124422
FoM: 0.01755650306718164, Std: 0.0025132063531738704
FoM: 0.02168843270088873, Std: 0.0044219754804381405
FoM: 0.03369014710649562, Std: 0.009667196574901437
FoM: 0.012956479239311303, Std: 0.005116175510444383
FoM: 0.019647468781457657, Std: 0.0017754922347877034
FoM: 0.01749282362366513, Std: 0.005459079822786908
FoM: 0.0504898477500442, Std: 0.009195672157945308
FoM: 0.04552667293354708, Std: 0.006122726337458457
FoM: 0.0274744275893975, Std: 0.008476075159791737
FoM: 0.01334326208819674, Std: 0.0045609876238289606
FoM: 0.044961764938338644, Std: 0.006283291846479946
FoM: 0.02382829044484526, Std: 0.007367291471575642
FoM: 0.011318233267909917, Std: 0.0026766424459521753
FoM: 0.002682792697403947, Std: 0.0075751679215327

FoM: 0.02702500346053293, Std: 0.005607626627157219
FoM: 0.019790612728715507, Std: 0.004224812535587286
FoM: 0.04587985892353415, Std: 0.0055790274741606485
FoM: 0.042213250719184504, Std: 0.002524917383491331
FoM: 0.036347335509748044, Std: 0.008833436734148957
FoM: 0.05936120951183609, Std: 0.005228440030279289
FoM: 0.03372624524216651, Std: 0.0023380720076661286
FoM: 0.005579233507808224, Std: 0.006652716785199003
FoM: 0.01682540038264311, Std: 0.003827494886788442
FoM: 0.04712038917214102, Std: 0.0034376495726951483
FoM: 0.027097996000186725, Std: 0.002566598799732699
FoM: 0.02283338343770685, Std: 0.0020639479967885844
FoM: 0.01847451779750634, Std: 0.001963219417391905
FoM: 0.00993035776722925, Std: 0.0077072344658758556
FoM: 0.003883865965646981, Std: 0.002837552480214015
FoM: 0.061344920035758234, Std: 0.00616322586337308
Too much time... Exit!
FoM: 0.017922355183839812, Std: 0.003122173508112166
Optimization finished


In [18]:
# Access the optimizer object to get info about the optimization
optimization_obj = optimizationlogic.optimization_obj
opt_alg_obj = optimization_obj.get_optimization_algorithm()

In [22]:
# Best FoM with std
print("FoM: {fom} +- {std}".format(fom=opt_alg_obj.best_FoM, std=opt_alg_obj.best_sigma))

FoM: 0.00679448742331189 +- 0.005813143003176931


In [28]:
# Best controls
best_controls_dict = opt_alg_obj.get_best_controls()
pulse_time = best_controls_dict["timegrids"][0]
pulse_amplitude = best_controls_dict["pulses"][0]
print(pulse_time)
print(pulse_amplitude)

[0.   0.03 0.06 0.09 0.12 0.15 0.18 0.21 0.24 0.27 0.3  0.33 0.36 0.39
 0.42 0.45 0.48 0.51 0.54 0.57 0.6  0.63 0.66 0.69 0.72 0.75 0.78 0.81
 0.84 0.87 0.9  0.93 0.96 0.99 1.02 1.05 1.08 1.11 1.14 1.17 1.2  1.23
 1.26 1.29 1.32 1.35 1.38 1.41 1.44 1.47 1.5  1.53 1.56 1.59 1.62 1.65
 1.68 1.71 1.74 1.77 1.8  1.83 1.86 1.89 1.92 1.95 1.98 2.01 2.04 2.07
 2.1  2.13 2.16 2.19 2.22 2.25 2.28 2.31 2.34 2.37 2.4  2.43 2.46 2.49
 2.52 2.55 2.58 2.61 2.64 2.67 2.7  2.73 2.76 2.79 2.82 2.85 2.88 2.91
 2.94 2.97 3.  ]
[ 2.66667466  2.28726972  1.93901163  1.62760984  1.35600014  1.12424816
  0.92961338  0.76677618  0.62822143  0.5047629   0.38618369  0.26195983
  0.12202759 -0.04244979 -0.23836234 -0.47002671 -0.73869524 -1.04223736
 -1.37502103 -1.7280109  -2.08908724 -2.44357619 -2.77496813 -3.06578838
 -3.29857322 -3.45689569 -3.52638031 -3.49564409 -3.35710346 -3.10759304
 -2.74875192 -2.28714633 -1.73411313 -1.10532538 -0.42009896  0.29952428
  1.02986636  1.74681357  2.4270566   3.04929589